In [2]:
import os
import sys

import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# dataset
src = o3d.io.read_point_cloud("src.ply")
dst = o3d.io.read_point_cloud("dst.ply")

Xall = np.asarray(src.points)
Yall = np.asarray(dst.points)

const_buf = np.ones((Xall.shape[0], 1))
Xall = np.concatenate((const_buf, Xall), axis=1) # 1+ local position
Yalls = np.hsplit(Yall, Yall.shape[1]) # independent global coordinate 

In [85]:
# recursive least squares

k = 3 # initial

X = Xall[:k]
# print(X)
# print(X.shape) # (k, 1+|features|)

dim_target = 0 # x, y, z
Y = Yalls[dim_target][:k] # TODO: do each RLS for each dimension!
# print(Y)
# print(Y.shape) # (k, 1)

P = np.linalg.inv(X.T @ X) # NOTE: inverse covariance matrix
# print(P)
# print(P.shape) # (1+|features|, 1+|features|)

beta = P @ X.T @ Y
# print(beta)
# print(beta.shape) # (1+|features|, 1), first dimension is intercept


for k in range(4, Xall.shape[0]):

    # print(k)

    z = Xall[k].reshape((Xall[k].shape[0], 1))
    y = Yalls[dim_target][k]
    

    G = (P @ z) / (1 + z.T @ P @ z) # weight
    # print(G)
    # print(G.shape) # (1, 1+|features|)
    
    P = P - ((G @ z.T) * P)
    # print(P)
    # print(P.shape) # (1+|features|, 1+|features|)

    beta = beta + G * (y - beta.T @ z)
    # print(beta)
    # print(beta.shape) # (1+|features|, 1)

    # break

print(beta)
idx_eval = 101
x_eval = Xall[idx_eval]
# print(x_eval)
# print(np.concatenate([x_eval[1:], x_eval[0]]))
# print(x_eval[1:])
# print(x_eval[0])
# print(np.array(x_eval[0]).reshape(1))
# print(np.concatenate([x_eval[1:], np.array(x_eval[0]).reshape(1)], axis=0))
x_eval = np.concatenate([x_eval[1:], np.array(x_eval[0]).reshape(1)], axis=0)

y_eval = Yalls[dim_target][idx_eval]
print(y_eval)
print(x_eval.dot(beta))


[[-29.96628819]
 [-37.67119207]
 [ 99.36610004]
 [-80.39318209]]
[-0.01247181]
[100.70198346]


In [88]:
# modulization check

from RLS_module import RLS

solver = RLS(X, Y)
# print(solver.beta)

for k in range(4, Xall.shape[0]):
    x = Xall[k]
    y = Yalls[dim_target][k]
    solver.add_new(x, y)

print(solver.beta)

idx_eval = 101
x_eval = Xall[idx_eval]
y_eval = Yalls[dim_target][idx_eval]
print(y_eval)
print(solver.evaluate(x_eval))

[[-29.96628819]
 [-37.67119207]
 [ 99.36610004]
 [-80.39318209]]
[-0.01247181]
[100.70198346]


In [91]:
# test with public data, 
# from https://github.com/adamDhalla/recursive-least-squares

# initialization
A = np.array([[1, 0], [1, 1], [1, 2]])
b = np.array([[3], [4], [7]])
solver = RLS(A, b)
print(f"x0={solver.beta}")

# update
newA = np.array([1, 3])
newb = np.array([11])
solver.add_new(newA, newb)
print(f"x1={solver.beta}")

x0=[[2.66666667]
 [2.        ]]
x1=[[2.2]
 [2.7]]
